<a href="https://colab.research.google.com/github/Adriano181/folio/blob/main/Ejerecicio_de_pruebas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# 1. Simulación de datos
np.random.seed(42)
n = 300
df = pd.DataFrame({
    'Edad': np.random.randint(18, 65, size=n),
    'FrecuenciaCompra': np.random.randint(1, 10, size=n),
    'GastoTotal': np.round(np.random.normal(800000, 200000, size=n), -3),
    'Ciudad': np.random.choice(['Bogotá', 'Tunja', 'Soacha', 'Chía'], size=n),
    'Servicio': np.random.choice(['Internet', 'Móvil', 'TV'], size=n)
})

# 2. Clasificación de NivelCliente
def clasificar(gasto):
    if gasto > 1000000:
        return 'Alto'
    elif gasto >= 600000:
        return 'Medio'
    else:
        return 'Bajo'

df['NivelCliente'] = df['GastoTotal'].apply(clasificar)

# 3. División de datos en entrenamiento y prueba (80% - 20%)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df = df.iloc[:240]
test_df = df.iloc[240:].copy()

# 4. Modelo simple basado en promedio de gasto por Ciudad y Servicio
promedios = train_df.groupby(['Ciudad', 'Servicio'])['GastoTotal'].mean().reset_index()

def predecir_nivel(fila):
    ciudad = fila['Ciudad']
    servicio = fila['Servicio']
    row = promedios[(promedios['Ciudad'] == ciudad) & (promedios['Servicio'] == servicio)]
    if not row.empty:
        gasto_est = row['GastoTotal'].values[0]
        return clasificar(gasto_est)
    else:
        return 'Medio'

# 5. Aplicar predicciones
test_df['Prediccion'] = test_df.apply(predecir_nivel, axis=1)

# 6. Matriz de confusión
etiquetas = ['Bajo', 'Medio', 'Alto']
matriz = pd.crosstab(
    test_df['NivelCliente'],
    test_df['Prediccion'],
    rownames=['Real'],
    colnames=['Predicho'],
    dropna=False
).reindex(index=etiquetas, columns=etiquetas, fill_value=0)

print("Matriz de Confusión:")
print(matriz)

# 7. Cálculo manual de métricas
print("\nReporte de Clasificación:")
for clase in etiquetas:
    TP = matriz.loc[clase, clase]
    FP = matriz[clase].sum() - TP
    FN = matriz.loc[clase].sum() - TP
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f"{clase:>5} -> Precisión: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")

# 8. Otras salidas útiles para pruebas automáticas (opcional)
print("Total de predicciones:", test_df['Prediccion'].notna().sum())
print("Tamaño del dataset:", df.shape)
print("Clases de NivelCliente:", df['NivelCliente'].unique())

Matriz de Confusión:
Predicho  Bajo  Medio  Alto
Real                       
Bajo         0     12     0
Medio        0     40     0
Alto         0      8     0

Reporte de Clasificación:
 Bajo -> Precisión: 0.00, Recall: 0.00, F1-Score: 0.00
Medio -> Precisión: 0.67, Recall: 1.00, F1-Score: 0.80
 Alto -> Precisión: 0.00, Recall: 0.00, F1-Score: 0.00
Total de predicciones: 60
Tamaño del dataset: (300, 6)
Clases de NivelCliente: ['Bajo' 'Medio' 'Alto']


Actividad: Análisis de Ventas de una Tienda 🛒
Una tienda ha registrado las ventas del último mes. Cada registro incluye:

👤 El nombre del cliente
📦 El producto que compró
🔢 La cantidad
💲 El precio unitario
📅 La fecha del pedido
🧠 Tareas del estudiante
📋 Crear un DataFrame con los datos proporcionados.
➗ Calcular el total por venta utilizando la fórmula: cantidad * precio_unitario.
📈 Agrupar los datos por cliente y calcular:
💰 El total gastado
🧾 El número de pedidos realizados
📊 Visualizar con un gráfico de barras el total gastado por cliente.
🏷️ Clasificar a los clientes en:
🔝 'Alto' si gastaron más de $1000
⚖️ 'Medio' si gastaron entre $500 y $1000
🔽 'Bajo' si gastaron menos de $500